# Introducción a NPL y extracción de características
Para poder aplicar cualquier algoritmo de NLP es necesario convertir el texto en caracteristicas numéricas (modelo *vector space*). Aquí vamos a ver las más comunes:  
- Modelo *Bag-of-Words* (BoW)  
- Modelo *Term Frequency-Inverse Document Frequency* (TF-IDF)  
- Modelo *Word Embeddings* (WE)  

Vamos a usar las librerías más comunes para esta tarea, `scikit-learn`, `gensim`, `spaCy`...

In [1]:
# librerías y opciones de entorno
%reset -sf
import pandas as pd
import numpy as np
import string
import spacy
from spacy.matcher import Matcher
import gensim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [2]:
# Instalamos la librería y el modelo de lenguaje para el español, solo la primera vez
!python -m spacy download es_core_news_lg

     -------------------------------------- 568.0/568.0 MB 4.9 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [2]:
# Cargamos el modelo de lenguaje para el español
nlp = spacy.load('es_core_news_lg')

In [3]:
# Se define el corpus que vamos a usar
corpus = [
    "El conocimiento es poder.",
    "La perseverancia es la clave del éxito.",
    "La creatividad es contagiosa, pásala.",
    "El viaje de mil millas comienza con un solo paso.",
    "La vida es lo que pasa mientras estás ocupado haciendo otros planes.",
    "El tiempo es un recurso no renovable, úsalo sabiamente.",
    "Nunca es tarde para ser lo que podrías haber sido."
]

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [4]:
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas que no son stop_words'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       not t.is_stop and
                       not t.is_space and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [5]:
# Aplicamos la función de normalización a todo el Corpus:
norm_corpus = [normalizar_doc(doc) for doc in corpus]
norm_corpus

['conocimiento',
 'perseverancia clave éxito',
 'creatividad contagiosa pásala',
 'viaje mil millas comienza paso',
 'vida pasa estás ocupado planes',
 'tiempo recurso renovable úsalo sabiamente',
 'podrías']

## Modelo Bag-of-Word (BoW)
El modelo BoW consiste en contar la frecuencia de aparición de cada término en todos los documentos, usando un diccionario de términos común.

In [6]:

cv = CountVectorizer()
cv.fit(norm_corpus)

CountVectorizer()

In [7]:
# distintas palabras
print(cv.get_feature_names_out())

['clave' 'comienza' 'conocimiento' 'contagiosa' 'creatividad' 'estás'
 'mil' 'millas' 'ocupado' 'pasa' 'paso' 'perseverancia' 'planes' 'podrías'
 'pásala' 'recurso' 'renovable' 'sabiamente' 'tiempo' 'viaje' 'vida'
 'éxito' 'úsalo']


In [8]:
# las convertimos en matriz
cv_matrix = cv.transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0],
       [0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0],
       [0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
        1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0]], dtype=int64)

In [9]:
# obtenemos palabras únicas en el corpus
vocab = cv.get_feature_names_out()
# mostramos vectores de características BoW del corpus
pd.DataFrame(cv_matrix, columns=vocab)

,clave,comienza,conocimiento,contagiosa,creatividad,estás,mil,millas,ocupado,pasa,...,podrías,pásala,recurso,renovable,sabiamente,tiempo,viaje,vida,éxito,úsalo
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


## Modelo N-grams
EL modelo BoW N-grams Considera como términos del vocabulario cada secuencia de N palabras consecutivas que aparece en el Corpus.  
Por ejemplo para los *bigrams* del corpus (N=2):

In [10]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names_out()
print(vocab_bigram)

['clave éxito' 'comienza paso' 'contagiosa pásala'
 'creatividad contagiosa' 'estás ocupado' 'mil millas' 'millas comienza'
 'ocupado planes' 'pasa estás' 'perseverancia clave' 'recurso renovable'
 'renovable úsalo' 'tiempo recurso' 'viaje mil' 'vida pasa'
 'úsalo sabiamente']


In [11]:
pd.DataFrame(bv_matrix, columns=vocab_bigram)

,clave éxito,comienza paso,contagiosa pásala,creatividad contagiosa,estás ocupado,mil millas,millas comienza,ocupado planes,pasa estás,perseverancia clave,recurso renovable,renovable úsalo,tiempo recurso,viaje mil,vida pasa,úsalo sabiamente
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Modelo TF-IDF
Este modelo promedia la frecuencia de aparición de cada término (*Term Frequency*) por el número de documentos en los que aparece (*Inverse Document Frequency*)

In [12]:
# se convierte el corpus en matriz
tv = TfidfVectorizer(norm=None)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix.shape

(7, 23)

La matriz resultante tiene los mismos términos que la del modelo BoW, pero se aplica un peso diferente a cada término en función de su frecuencia de aparición en distintos documentos:

In [13]:
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names_out()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,clave,comienza,conocimiento,contagiosa,creatividad,estás,mil,millas,ocupado,pasa,...,podrías,pásala,recurso,renovable,sabiamente,tiempo,viaje,vida,éxito,úsalo
0,0.00,0.00,2.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,2.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.39,0.00
2,0.00,0.00,0.00,2.39,2.39,0.00,0.00,0.00,0.00,0.00,...,0.00,2.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,2.39,0.00,0.00,0.00,0.00,2.39,2.39,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,2.39,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,2.39,0.00,0.00,2.39,2.39,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2.39,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,2.39,2.39,2.39,2.39,0.00,0.00,0.00,2.39
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,2.39,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## Word embeddings con spaCy
Los word embeddings (o word vectors) son representaciones numéricas de las palabras, generadas con una reducción de dimensionalidad sobre una matriz de co-ocurrencia sobre un corpus enorme. Spacy utiliza los word vectors de GloVe, (*Stanford's Global Vectors for Word Representation*). Estos vectores se pueden utilizar para calcular la similaridad semántica entre palabras o documentos.

El vocabulario por defecto en el modelo spaCy del idioma inglés (`en_core_web_sm`) es muy pequeño. Hay que cargar en_core_web_md (`python -m spacy download en_core_web_md`) para tener un conjunto de word vectors mayor. El modelo de tamaño medio en español (`python -m spacy download es_core_news_md`) contiene vectores también.

#### Introducción a la librería spaCy


##### Procesado de texto

spaCy ejecuta todos los análisis del texto con una sola instrucción. Esta instrucción ejecuta un *pipeline* (procesado secuencial) que implementa:  

- División en tokens  
- Lematizado
- Análisis gramatical
- Análisis de dependencias
- *Name Entity Recognition* (NER)

Vamos a ver estas funcionalidades previamente a introducir el word embeddings

In [14]:
# Ejemplo de texto
texto = "El perro de San Roque es muy agresivo."

In [15]:
# Lo primero que hacemos es analizar el texto y generar un objeto de tipo `Doc`
parsedData = nlp(texto)
type(parsedData)

spacy.tokens.doc.Doc

In [16]:
datos = map(lambda t: {'token': t.orth_,
                       'lema': t.lemma_,
                       'shape': t.shape_,
                       'POS': t.pos_,
                       'POS detallado': t.tag_,
                       'dependencia': t.dep_,
                       'Descripción dep': spacy.explain(t.dep_)}, parsedData)

pd.DataFrame(datos)

,token,lema,shape,POS,POS detallado,dependencia,Descripción dep
0,El,el,Xx,DET,DET,det,determiner
1,perro,perro,xxxx,PROPN,PROPN,nsubj,nominal subject
2,de,de,xx,ADP,ADP,case,case marking
3,San,San,Xxx,PROPN,PROPN,flat,flat multiword expression
4,Roque,Roque,Xxxxx,PROPN,PROPN,flat,flat multiword expression
5,es,ser,xx,AUX,AUX,cop,copula
6,muy,mucho,xxx,ADV,ADV,advmod,adverbial modifier
7,agresivo,agresivo,xxxx,ADJ,ADJ,ROOT,root
8,.,.,.,PUNCT,PUNCT,punct,punctuation


###### Exploramos el documento

In [17]:
# Al analizar un texto, spaCy lo divide en una lista de `tokens`, que se acceden iterando sobre el objeto `Doc`
[t for t in parsedData]

[El, perro, de, San, Roque, es, muy, agresivo, .]

Las propiedades más importantes son (https://spacy.io/api/token#attributes):  
* `orth_`: texto del token
* `lemma_`: lema (palabra base)
* `shape_`: forma ortográfica del token
* `pos_`: Part-of-Speech (genérico)
* `tag_`: POS detallado
* `dep_`: Tipo de dependencia del token (análisis de dependencias)

##### Análisis gramatical
Los documentos SpaCy también dividen en texto en oraciones (*sentences*) que son objetos del tipo `spacy.tokens.span.Span`. Podemos iterar con el generador `doc.sents` usando `next()`, `list()`, un bucle o con una comprensión de lista.

In [18]:
texto = """
La Inteligencia Artificial (IA) es una de las revoluciones tecnológicas más significativas de nuestro tiempo. Este campo multidisciplinario de la informática se enfoca en desarrollar sistemas y algoritmos que pueden realizar tareas que, en el pasado, requerían la intervención humana y el razonamiento. La IA busca imitar y automatizar el pensamiento humano, lo que abre un mundo de posibilidades en diversas industrias y aspectos de la vida cotidiana.

Uno de los hitos más destacados de la IA es el aprendizaje automático (Machine Learning), que permite a las máquinas aprender y mejorar su rendimiento a partir de datos y experiencias previas. Esto ha llevado al desarrollo de sistemas capaces de tomar decisiones informadas, como los vehículos autónomos, que pueden conducir de manera segura y eficiente, o los sistemas de recomendación en plataformas de streaming, que sugieren contenido personalizado.

En la medicina, la IA ha demostrado su valía en la detección temprana de enfermedades, la interpretación de imágenes médicas y la optimización de tratamientos. En la atención al cliente, los chatbots y asistentes virtuales proporcionan respuestas instantáneas y ayudan a resolver consultas.

La IA también tiene un impacto importante en la industria manufacturera, donde la automatización y la robótica inteligente mejoran la eficiencia y la calidad de la producción. Además, en la investigación científica, la IA ayuda a analizar grandes conjuntos de datos y a realizar descubrimientos importantes.

Sin embargo, la IA no está exenta de desafíos. Se plantean cuestiones éticas, como la privacidad de los datos y la toma de decisiones algorítmicas. También se debate sobre el posible impacto en el empleo, ya que algunas tareas humanas pueden ser automatizadas.

En resumen, la Inteligencia Artificial es una fuerza impulsora que está transformando radicalmente la forma en que vivimos y trabajamos. A medida que continúa avanzando, es esencial abordar sus desafíos éticos y sociales, al tiempo que se aprovecha su potencial para mejorar la eficiencia, la precisión y la calidad de vida en todo el mundo. La IA está moldeando el futuro y promete seguir sorprendiéndonos con sus innovaciones y aplicaciones aún no descubiertas."""
parsedData = nlp(texto)
parsedData


La Inteligencia Artificial (IA) es una de las revoluciones tecnológicas más significativas de nuestro tiempo. Este campo multidisciplinario de la informática se enfoca en desarrollar sistemas y algoritmos que pueden realizar tareas que, en el pasado, requerían la intervención humana y el razonamiento. La IA busca imitar y automatizar el pensamiento humano, lo que abre un mundo de posibilidades en diversas industrias y aspectos de la vida cotidiana.

Uno de los hitos más destacados de la IA es el aprendizaje automático (Machine Learning), que permite a las máquinas aprender y mejorar su rendimiento a partir de datos y experiencias previas. Esto ha llevado al desarrollo de sistemas capaces de tomar decisiones informadas, como los vehículos autónomos, que pueden conducir de manera segura y eficiente, o los sistemas de recomendación en plataformas de streaming, que sugieren contenido personalizado.

En la medicina, la IA ha demostrado su valía en la detección temprana de enfermedades, la 

In [19]:
# oraciones
for i, sent in enumerate(parsedData.sents):
    print("Oración {}:\n{}\n".format(i,sent))

Oración 0:



Oración 1:
La Inteligencia Artificial (IA) es una de las revoluciones tecnológicas más significativas de nuestro tiempo.

Oración 2:
Este campo multidisciplinario de la informática se enfoca en desarrollar sistemas y algoritmos que pueden realizar tareas que, en el pasado, requerían la intervención humana y el razonamiento.

Oración 3:
La IA busca imitar y automatizar el pensamiento humano, lo que abre un mundo de posibilidades en diversas industrias y aspectos de la vida cotidiana.



Oración 4:
Uno de los hitos más destacados de la IA es el aprendizaje automático (Machine Learning), que permite a las máquinas aprender y mejorar su rendimiento a partir de datos y experiencias previas.

Oración 5:
Esto ha llevado al desarrollo de sistemas capaces de tomar decisiones informadas, como los vehículos autónomos, que pueden conducir de manera segura y eficiente, o los sistemas de recomendación en plataformas de streaming, que sugieren contenido personalizado.



Oración 6:
En l

###### Part of Speech (POS)
La librería `spaCy` determina el tipo gramatical (POS) de cada palabra en nuestro texto. Creamos un diccionario con los distintos POS de nuestro texto de ejemplo, usando el *hash* de cada POS como clave del diccionario:

In [20]:
{w.pos: (w.pos_, spacy.explain(w.pos_)) for w in parsedData}

{103: ('SPACE', 'space'),
 90: ('DET', 'determiner'),
 96: ('PROPN', 'proper noun'),
 97: ('PUNCT', 'punctuation'),
 87: ('AUX', 'auxiliary'),
 95: ('PRON', 'pronoun'),
 85: ('ADP', 'adposition'),
 92: ('NOUN', 'noun'),
 84: ('ADJ', 'adjective'),
 86: ('ADV', 'adverb'),
 100: ('VERB', 'verb'),
 89: ('CCONJ', 'coordinating conjunction'),
 98: ('SCONJ', 'subordinating conjunction')}

Cada tipo gramatical (POS) se subdivide en distintas etiquetas (POS detallado o tag).
Por ejemplo en nuestro texto tenemos los siguientes tag:

In [21]:
set([(w.pos_, w.tag_) for w in parsedData])

{('ADJ', 'ADJ'),
 ('ADP', 'ADP'),
 ('ADV', 'ADV'),
 ('AUX', 'AUX'),
 ('CCONJ', 'CCONJ'),
 ('DET', 'DET'),
 ('NOUN', 'NOUN'),
 ('PRON', 'PRON'),
 ('PROPN', 'PROPN'),
 ('PUNCT', 'PUNCT'),
 ('SCONJ', 'SCONJ'),
 ('SPACE', 'SPACE'),
 ('VERB', 'VERB')}

###### Análisis de dependencias (dependency parsing)
La librería `spaCy`también analiza las relaciones entre palabras de una frase.

In [22]:
texto2 = "La Alhambra de Granada es un monumento muy bonito."

parsedData2 = nlp(texto2)
dependencias = [(t.text, t.dep_, spacy.explain(t.dep_)) for t in parsedData2]
pd.DataFrame(list(dependencias), columns=['texto', 'dependencia', 'explicación'])

,texto,dependencia,explicación
0,La,det,determiner
1,Alhambra,nsubj,nominal subject
2,de,case,case marking
3,Granada,flat,flat multiword expression
4,es,cop,copula
5,un,det,determiner
6,monumento,ROOT,root
7,muy,advmod,adverbial modifier
8,bonito,amod,adjectival modifier
9,.,punct,punctuation


In [23]:
# Análisis a cada palabra del texto.
sent=next(parsedData2.sents)
for word in sent:
    print(word, ': ', str(list(word.children)))

La :  []
Alhambra :  [La, Granada]
de :  []
Granada :  [de]
es :  []
un :  []
monumento :  [Alhambra, es, un, bonito, .]
muy :  []
bonito :  [muy]
. :  []


Analizamos las dependencias de cada palabra para una sentencia. Cada palabra tiene una palabra de la que depende (.head) y unas palabras que dependen de ella (children) a izquierda (.lefts) y a derecha (.rights).

In [24]:
print("Palabra: [tipo de dependencia] HEAD")
print("-----------------------------------")
for token in parsedData2:
    print("{}: [{}({})] |HEAD:{}".format(token.orth_, token.dep_, spacy.explain(token.dep_), token.head.orth_))

print("\n[Dependencias izquierdas]<---palabra[tipo de dependencia]--->[dependencias derechas]")
for token in parsedData2:
    print("{left}<---{word}[{dep_tag}]--->{right}\n-----".format(word=token.orth_,
        dep_tag=token.dep_, left=[t.orth_ for t in token.lefts],
        right=[t.orth_ for t in token.rights]))

Palabra: [tipo de dependencia] HEAD
-----------------------------------
La: [det(determiner)] |HEAD:Alhambra
Alhambra: [nsubj(nominal subject)] |HEAD:monumento
de: [case(case marking)] |HEAD:Granada
Granada: [flat(flat multiword expression)] |HEAD:Alhambra
es: [cop(copula)] |HEAD:monumento
un: [det(determiner)] |HEAD:monumento
monumento: [ROOT(root)] |HEAD:monumento
muy: [advmod(adverbial modifier)] |HEAD:bonito
bonito: [amod(adjectival modifier)] |HEAD:monumento
.: [punct(punctuation)] |HEAD:monumento

[Dependencias izquierdas]<---palabra[tipo de dependencia]--->[dependencias derechas]
[]<---La[det]--->[]
-----
['La']<---Alhambra[nsubj]--->['Granada']
-----
[]<---de[case]--->[]
-----
['de']<---Granada[flat]--->[]
-----
[]<---es[cop]--->[]
-----
[]<---un[det]--->[]
-----
['Alhambra', 'es', 'un']<---monumento[ROOT]--->['bonito', '.']
-----
[]<---muy[advmod]--->[]
-----
['muy']<---bonito[amod]--->[]
-----
[]<---.[punct]--->[]
-----


In [25]:
from spacy import displacy

displacy.render(parsedData2, style='dep', jupyter=True, options={'distance':120})

###### Búsqueda de patrones
Spacy tiene una clase Matcher que permite buscar tokens con un patrón definido en los objetos Doc. Se puede buscar por el texto del token o por atributos del token.

In [25]:
from spacy.matcher import Matcher

#inicializamos sobre el vocabulario
matcher = Matcher(nlp.vocab)

In [26]:
#definimos un patrón de texto a buscar
patron = [{"TEXT": "iPhone"}, {"TEXT": "X"}] #Patrón: texto 'iPhone' seguido de texto 'X'
matcher.add("iphone_x", None, patron)

#procesamos un documento con el patrón
doc = nlp("El iPhone X salió después del iPhone 8, pero nunca sacaron el iPhone 9")

#llamamos al matcher
matches = matcher(doc)

#iteramos sobre los resultados
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

TypeError: add() takes exactly 2 positional arguments (3 given)

###### Análisis de dependencias (dependency parsing)
La librería `spaCy`también analiza las relaciones entre palabras de una frase.

In [26]:
#inicializamos matcher
matcher = Matcher(nlp.vocab)
#definimos un patrón
patron = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("nombre+adjetivo", [patron])

#procesamos un texto con el patron
doc = nlp(texto)
#llamamos al matcher
matches = matcher(doc)

#iteramos sobre el resultado
for match_id, start, end in matches:
    matched_span = doc[start:end]
    print(matched_span.text)

revoluciones tecnológicas
campo multidisciplinario
intervención humana
pensamiento humano
vida cotidiana
aprendizaje automático
experiencias previas
sistemas capaces
decisiones informadas
vehículos autónomos
manera segura
contenido personalizado
detección temprana
imágenes médicas
asistentes virtuales
respuestas instantáneas
impacto importante
industria manufacturera
robótica inteligente
investigación científica
descubrimientos importantes
cuestiones éticas
decisiones algorítmicas
tareas humanas
fuerza impulsora
desafíos éticos


##### Explorando las entidades propias (Named Entities)
La librería `spaCy` determina las entidades propias que aparecen en el texto. Podemos acceder a las entidades de un documento a través de su atributo `doc.ents`.  
Por ejemplo en el artículo cargado hay las siguientes entidades:

In [27]:
texto_raw = """La relación entre Madrid y Barcelona es una de las más intrigantes y complejas en España, y de hecho, en toda Europa. Estas dos ciudades icónicas tienen historias y personalidades distintas, pero también comparten muchos aspectos en común.
Madrid, como la capital de España, desempeña un papel central en la vida política, económica y cultural del país. Es conocida por su vitalidad, su bullicioso ambiente y su rica historia. Barcelona, por otro lado, es la capital de Cataluña y tiene su propia identidad única. Es famosa por su arquitectura modernista, sus playas mediterráneas y su cultura cosmopolita.
A lo largo de la historia, Madrid y Barcelona han competido en diversos aspectos, desde el deporte hasta la economía y la influencia política. Una de las rivalidades más intensas se encuentra en el fútbol, donde el clásico enfrentamiento entre el Real Madrid y el FC Barcelona es seguido con pasión en todo el mundo. Esta rivalidad refleja en parte las tensiones históricas entre las regiones de Castilla (donde se encuentra Madrid) y Cataluña.
No obstante, en los últimos años, la relación entre Madrid y Barcelona ha evolucionado. Ambas ciudades son motores económicos de España y, en muchos aspectos, se complementan mutuamente. Madrid es el centro financiero y político, mientras que Barcelona es líder en la industria creativa, el turismo y la innovación. Muchas empresas tienen presencia en ambas ciudades, lo que ha contribuido a una mayor interconexión.
En términos de cultura, Madrid y Barcelona son centros culturales vibrantes y atractivos para artistas, músicos y amantes de las artes en general. Ambas ciudades albergan museos e renombre mundial, teatros y eventos culturales que enriquecen la vida de sus habitantes y visitantes.
A nivel turístico, Madrid y Barcelona son dos de los destinos más populares de España. Los viajeros pueden disfrutar de la belleza arquitectónica de Barcelona, con obras maestras como la Sagrada Familia de Gaudí, y luego sumergirse en la historia y la gastronomía de Madrid, con su impresionante Palacio Real y sus famosos mercados de alimentos.
En resumen, la relación entre Madrid y Barcelona es una mezcla de competencia histórica y cooperación contemporánea. Estas dos ciudades emblemáticas de España, con sus diferencias y similitudes, siguen desempeñando un papel fundamental en la identidad y la cultura del país, contribuyendo a su diversidad y riqueza. La relación entre ellas es un reflejo de la complejidad y la dinámica de España como nación.
"""
doc = nlp(texto_raw)


In [28]:
# El atributo `doc.ents` devuelve una tupla con las entidades del documento (objetos de tipo `Span`)
entities = [(e.text, e.label_) for e in doc.ents]
pd.DataFrame(list(entities), columns=['entidad', 'tipo']).drop_duplicates()


,entidad,tipo
0,Madrid,LOC
1,Barcelona,LOC
2,España,LOC
3,Europa,LOC
7,Cataluña,LOC
10,Una de las rivalidades más intensas,MISC
11,Real Madrid,ORG
12,FC Barcelona,ORG
13,Castilla,LOC
26,Los viajeros pueden disfrutar de la belleza arquitectónica de Barcelona,MISC


In [29]:
# Los tipos de entidades que aparecen en el documento son:
{w.label_:spacy.explain(w.label_) for w in doc.ents}

{'LOC': 'Non-GPE locations, mountain ranges, bodies of water',
 'MISC': 'Miscellaneous entities, e.g. events, nationalities, products or works of art',
 'ORG': 'Companies, agencies, institutions, etc.',
 'PER': 'Named person or family.'}

In [30]:
# Podemos visualizar gráficamente las entidades en su contexto con displaCy:
displacy.render(doc, style='ent', jupyter=True)

#### Word embeddings

In [31]:
# generamos varios vectores
madrid = nlp.vocab["Madrid"]
barcelona = nlp.vocab["Barcelona"]
granada = nlp.vocab["Granada"]
madrid.vector

array([ 1.9316e-01, -1.9055e+00,  1.1441e+00, -4.5437e-01, -7.4829e-01,
        3.8813e-01, -2.3510e+00, -1.6627e+00,  1.7095e+00, -1.8088e-01,
        1.2363e+00, -9.0759e-02,  5.6769e-01,  2.2311e+00, -1.0389e+00,
       -2.6093e+00, -3.0297e-01,  1.3868e+00, -2.5877e+00, -1.0528e+00,
       -2.5049e+00,  7.1522e-01, -2.0612e+00,  9.4640e-01,  8.0187e-01,
       -5.3548e-01, -2.5208e-01, -2.8305e-01,  2.3178e-01, -5.9537e-01,
       -5.0337e-01, -9.7432e-01,  8.2222e-01, -2.3384e+00,  2.1131e+00,
       -1.4344e+00,  2.0885e+00, -1.3344e+00,  4.8458e-01,  9.5693e-01,
       -1.0339e+00,  2.1483e+00, -3.3474e+00,  1.1917e+00,  6.2875e-01,
        1.1625e+00,  3.7431e-01, -2.3933e+00, -1.2356e-01,  8.3158e-01,
       -2.6937e+00, -1.7771e+00,  3.8745e-01,  1.9141e+00, -1.9752e-01,
       -2.0079e+00, -1.0141e+00,  6.3490e-02, -5.8045e-01,  1.9114e+00,
        2.2471e+00,  3.0757e+00,  3.3882e-01, -2.6576e-01,  1.4031e+00,
        1.5642e+00, -2.2871e+00,  1.6472e+00,  8.1172e-02, -1.52

In [32]:
# similitudes entre vectores
print(madrid.similarity(barcelona))
print(madrid.similarity(granada))
print(barcelona.similarity(granada))

0.8017494082450867
0.6647977828979492
0.6088482141494751


In [33]:
# Obtenemos las n palabras más cercanas a: turismo
turismo = nlp.vocab['turismo']

# cogemos todas las palabras del vocabulario que tienen vector, en minúsculas
allWords = list({w for w in nlp.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "turismo"})

print("longitud:",len(allWords))

# ordenamos por similitud con turismo
allWords.sort(key=lambda w: turismo.similarity(w))
allWords.reverse()
print("Top 20 palabras más similares a turismo:")
for word in allWords[:20]:
    print(word.orth_)

longitud: 451
Top 20 palabras más similares a turismo:
turístico
gastronomía
deporte
cultural
economía
desarrollo
cultura
innovación
económica
culturales
industria
destinos
empleo
ambiente
diversidad
potencial
industrias
capital
atractivos
viajeros
